In [2]:
import cv2
import tensorflow as tf
import numpy as np
from keras import models, layers, callbacks
from keras.preprocessing import image
import os
from prettytable import PrettyTable

In [4]:
model= models.load_model('modelGenGroup1.h5')

In [5]:
def color_segment_function(img_array):
    img_array= np.rint(img_array)
    img_array= img_array.astype('uint8')
    hsv_img= cv2.cvtColor(img_array, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv_img, (24, 50, 0), (55, 255, 255))
    result = cv2.bitwise_and(img_array, img_array, mask=mask)
    result= result.astype('float64')
    return result


In [6]:
#image function from keras.preprocessing
train_datagen = image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.0,
      height_shift_range=0.0,
      shear_range=0.0,
      zoom_range=0.0,
      horizontal_flip=True,
      vertical_flip= True,
    preprocessing_function=color_segment_function,
      fill_mode='nearest')

test_datagen = image.ImageDataGenerator(rescale=1./255, preprocessing_function=color_segment_function)

In [7]:
path ='/home/mike/Downloads/Classification'
test_ = os.path.join(path, 'test')
test_generator = test_datagen.flow_from_directory(
        test_,
        target_size=(150,150),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

Found 900 images belonging to 1 classes.


In [8]:
pred= model.predict_generator(test_generator, steps= test_generator.n, verbose=1)

/tmp/ipykernel_30465/4246402030.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred= model.predict_generator(test_generator, steps= test_generator.n, verbose=1)


900/900 [==============================] - 14s 15ms/step


In [9]:
label_map = {0: 'black_grass',
 1: 'charlock',
 2: 'cleavers',
 3: 'common_chickweed',
 4: 'common_wheat',
 5: 'fat_hen',
 6: 'loose_silky_bent',
 7: 'maize',
 8: 'scentless_mayweed',
 9: 'shepherds_purse',
 10: 'small_flowered_cranesbill',
 11: 'sugar_beet'}

In [10]:
predicted_class_indices=np.argmax(pred,axis=1)

prediction_labels = [label_map[k] for k in predicted_class_indices]
filenames= test_generator.filenames

In [11]:
import csv
csvfile= open('group1detections.csv', 'w', newline='')
writer= csv.writer(csvfile)

headers= ['file', 'species']

writer.writerow(headers)
t = PrettyTable(headers)
for i, f, p in zip(range(len(filenames)), filenames, prediction_labels):
    writer.writerow([os.path.basename(f),p])
    if i <10:
        t.add_row([os.path.basename(f), p])
    elif i<13:
        t.add_row(['.', '.'])
csvfile.close()
print(t)

+---------+-------------+
|   file  |   species   |
+---------+-------------+
| 000.png | black_grass |
| 001.png | black_grass |
| 002.png | black_grass |
| 003.png | black_grass |
| 004.png | black_grass |
| 005.png | black_grass |
| 006.png | black_grass |
| 007.png | black_grass |
| 008.png | black_grass |
| 009.png | black_grass |
|    .    |      .      |
|    .    |      .      |
|    .    |      .      |
+---------+-------------+


In [ ]:
file = open('results.txt', 'x')
file.write('id;label\n')
for i in range (len(prediction_labels)):
    if(i<10):
        file.write('00'+str(i)+';'+prediction_labels[i]+'\n')
    elif(1<100):
        file.write('0'+str(i)+ ';'+prediction_labels[i]+'\n')
    else:
        file.write(str(i)+';'+prediction_labels[i]+'\n')